In [13]:
import os
import re
from datetime import datetime
import fitz
THAI_MONTH_MAP = {
    "มกราคม": "01", "กุมภาพันธ์": "02", "มีนาคม": "03", "เมษายน": "04",
    "พฤษภาคม": "05", "มิถุนายน": "06", "กรกฎาคม": "07", "สิงหาคม": "08",
    "กันยายน": "09", "ตุลาคม": "10", "พฤศจิกายน": "11", "ธันวาคม": "12"
}

In [29]:
def extract_be_year_and_month(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for i in range(min(3, len(doc))):
        text += doc[i].get_text()

    # Match: month + optional ปี + optional พ.ศ. + year
    pattern = r"(" + "|".join(THAI_MONTH_MAP.keys()) + r")\s*(ปี)?\s*(พ\.ศ\.)?\s*(\d{2,4})"
    match = re.search(pattern, text)
    if not match:
        raise ValueError("❌ ไม่พบเดือนหรือปีในเนื้อหา PDF")

    thai_month = match.group(1)
    raw_year = int(match.group(4))

    # Always treat as BE year
    be_year = raw_year if raw_year > 100 else 2500 + raw_year
    month_number = THAI_MONTH_MAP[thai_month]

    return str(be_year), month_number

In [27]:
def create_nested_folder(base_path, be_year, month_number):
    folder_path = os.path.join(base_path, be_year, month_number)
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

In [30]:
be_year, month = extract_be_year_and_month("สลีป พ.ค.68.pdf")
output_folder = create_nested_folder('/', be_year, month)